In [3]:
import tensorflow as tf
import matplotlib.image as mpimg
import numpy as np
import os
import matplotlib.pyplot as plt
from math import pi
from tensorflow import gfile
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.python.saved_model import tag_constants
import time

C:\Program Files11\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# Путь к папке с данными, которые находятся в виде картинок и сгрупированы по папкам
# с числами, к которыми эти картинки являются.
# Этот путь используется в методе для улучшения нашего набора данных
data_path = "C:/Users/pyroman/dataset/"
epoch = 50
label_to_index_map = {}
IMAGE_SIZE = 100
learning_rate = 0.0001
batch_size = 100
max_checks_without_progress = 15

In [3]:
#Перегон массивов numpy в tf.tensor для дальнейших преобразований
def get_imgs_tf(imgs) :
    X_tf = []
    for img in imgs :
        tf_img = tf.convert_to_tensor(img)
        X_tf.append(tf_img)
    return X_tf

In [4]:
#Сохранение каждой картинки в виде массива numpy с сохранием одного канала
def saving_data(data, label, data_path=data_path) :
    global aug_data_path
    aug_data_path = os.path.join(data_path, "aug_data")
    aug_path_imgs = os.path.join(aug_data_path, label)
    os.makedirs(aug_path_imgs)
    i = 0
    for img in data :
        i+=1
        img = np.squeeze(img, 2)
        np.save(aug_path_imgs + "/"+str(i)+".npy", img)

In [5]:
#Улучшение набора данных путем увеличения количества тренировочных/проверочных
#что достигается путем многократных поворотов на некоторые углы и отзеркаливания
def get_aug_imgs(RGBimgs, label) :  
    imgs = []
    
    tf.reset_default_graph()
    with tf.Session() as sess:
        for img in RGBimgs :
            convert = tf.image.rgb_to_grayscale(img)
            conv_img = sess.run(convert)
            resize = tf.image.resize_images(conv_img, [88, 88])
            imgs.append(sess.run(resize))
            
    tf_imgs = get_imgs_tf(imgs)
    with tf.Session() as sess :
        for img in tf_imgs :
            aug3rd = tf.contrib.image.rotate(img, 15 * pi / 180)
            third_stage_aug = sess.run(aug3rd)
            imgs.append(third_stage_aug)
            
    tf.reset_default_graph()         
    tf_imgs = get_imgs_tf(imgs)
    with tf.Session() as sess :
        for img in tf_imgs :
            aug4th = tf.contrib.image.rotate(img, 335 * pi / 180)
            fourth_stage_aug = sess.run(aug4th)
            imgs.append(fourth_stage_aug)
            
    #В случае римской четверки и шестерки, после отзеркаливания они будут совпадать,
    #поэтому вместо отзеркаливания я делаю лишний поворот
    if (label == '4') | (label == '6') :
        tf.reset_default_graph()         
        tf_imgs = get_imgs_tf(imgs)
        with tf.Session() as sess :
            for img in tf_imgs :
                aug4th = tf.contrib.image.rotate(img, 20 * pi / 180)
                fourth_stage_aug = sess.run(aug4th)
                imgs.append(fourth_stage_aug)
    else :
        tf.reset_default_graph()
        tf_imgs = get_imgs_tf(imgs)
        with tf.Session() as sess :
            for img in tf_imgs :
                aug1st = tf.image.flip_left_right(img)
                first_stage_aug = sess.run(aug1st)
                imgs.append(first_stage_aug)
            
    tf.reset_default_graph()         
    tf_imgs = get_imgs_tf(imgs)
    with tf.Session() as sess :
        for img in tf_imgs :
            aug2nd = tf.contrib.image.rotate(img, 40 * pi / 180)
            second_stage_aug = sess.run(aug2nd)
            imgs.append(second_stage_aug)
            
    tf.reset_default_graph()         
    tf_imgs = get_imgs_tf(imgs)
    with tf.Session() as sess :
        for img in tf_imgs :
            aug5th = tf.contrib.image.rotate(img, 310 * pi / 180)
            fifth_stage_aug = sess.run(aug5th)
            imgs.append(fifth_stage_aug)
    
    tf.reset_default_graph()         
    tf_imgs = get_imgs_tf(imgs)
    with tf.Session() as sess :
        for img in tf_imgs :
            aug6th = tf.contrib.image.rotate(img, 15 * pi / 180)
            sixth_stage_aug = sess.run(aug6th)
            imgs.append(sixth_stage_aug)
    
    return imgs

In [6]:
#Основной метод для улучшения тренировочных и проверочных данных.
#На вход подается путь к папке с данными, которые должны находится
#В виде папка(имя папки - число) и внутри изображения формата .jpg
def get_aug_data(path) :
    labels = os.listdir(path)
    index = 0
    for label in labels :
        label_to_index_map[label] = index
        print(label_to_index_map)
        index+=1
        X= []
        path_folder = os.path.join(path, label)
        names_img = os.listdir(path_folder)
        for img in names_img :
            X_img = os.path.join(path_folder, img)
            image = mpimg.imread(X_img)
            X.append(image)
            
        aug_imgs = get_aug_imgs(X, label)
        saving_data(aug_imgs, label)

In [7]:
#Реализация one-hot encoding для разметки
def encoding_labels(label) :
    encoding = [0] * len(label_to_index_map)
    encoding[label_to_index_map[label]] = 1
    return encoding

In [8]:
#Способ загрузки данных, которые находятся в формате .npy
#Также этот метод разбивает данные на тренировочные и проверочные
#в отношении (1-k)/k и возвращает сразу готовые данные(данные и разметка)
def get_train_and_test_data(path, k) :
    y = []
    X = []
    path = os.path.join(path, '*', '*.npy')
    npimgs = gfile.Glob(path)
    i=0
    for npimg in npimgs :
        i+=1
        _, label = os.path.split(os.path.dirname(npimg))
        img = np.load(npimg)
        X.append(img)
        y.append(encoding_labels(label))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = k, random_state = 42)   
        
    return X_train, y_train, X_test, y_test

In [29]:
#Сохраняет приготовленные для тренировки и проверки данные в формате numpy  массивов
#по указаному пути path
def save_prepared_data(X_train, y_train, X_test, y_test, path) :
    np.save(os.path.join(path, "train_data.npy"), X_train)
    np.save(os.path.join(path, "train_labels.npy"), y_train)
    np.save(os.path.join(path, "test_data.npy"), X_test)
    np.save(os.path.join(path, "train_labels.npy"), y_test)

In [2]:
def load_prepared_data(path) :
    X_train = np.load(os.path.join(path, "train_data.npy"))
    y_train = np.load(os.path.join(path, "train_labels.npy"))
    X_test = np.load(os.path.join(path, "test_data.npy"))
    y_test = np.load(os.path.join(path, "test_labels.npy"))
    return X_train, y_train, X_test, y_test

In [15]:
#Определяем нашу нейронную сеть:
# Первый сверточный слой принимает на вход массив 88х88, применяет 8 фильтров 5х5
# с функцией активации ReLU
# Первый объеденяющий слой по максимуму со страйдом 2
# Второй сверточный слой принимает на вход массив 44х44, применяет 16 фильтров 5х5
# с функцией активации ReLU
# Второй объеденяющий слой по максимуму со страйдом 2
# Третий сверточный слой принимает на вход массив 22х22, применяет 32 фильтра 5х5
# с функцией активации ReLU
# Третий объеденяющий слой по максимуму со страйдом 2
# Первый полносвязный слой со входным количесвтом нейронов 3872 и выходным 3000
# с функцией активации ReLU
# Первый полносвязный слой со входным количесвтом нейронов 3000 и выходным 1500
# с функцией активации ReLU
# Второй полносвязный слой со входным количесвтом нейронов 1500 и выходным 700
# с функцией активации ReLU и dropout rate, который можно задать самостоятельно(моя модель имела dropout 0.5 при тренировке)
# Третий полносвязный слой со входным количесвтом нейронов 750 и выходным 100
# с функцией активации ReLU и dropout rate, который можно задать самостоятельно(моя модель имела dropout 0.5 при тренировке)
# Logits слой, с выходным количесвтом нейронов 8, 1 на каждый класс


def cnn_model(features, dropout) :
  # Input Layer
    input_layer = tf.reshape(features, [-1, 88, 88, 1])
    
    print(type(input_layer))

  #Первый сверточный слой
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=16,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    conv3 = tf.layers.conv2d(
      inputs=pool2,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

  # Dense Layer
    pool2_flat = tf.reshape(pool3, [-1, 11 * 11 * 32])
    dense = tf.layers.dense(inputs=pool2_flat, units=3000, activation=tf.nn.relu)
    
    dense1 = tf.layers.dense(inputs=dense, units=1500, activation=tf.nn.relu)
    
    dense2 = tf.layers.dense(inputs=dense1, units=750, activation=tf.nn.relu)
    
    drop2 = tf.layers.dropout(inputs = dense2, rate = dropout)
    
    dense3 = tf.layers.dense(inputs=drop2, units=100, activation=tf.nn.relu)
    
  # Logits Layer
    logits = tf.layers.dense(inputs=dense3, units=8)
    return logits

In [25]:
# Тренировка нейронной сети, на вход подаются уже готовые данные, на выходе показывается среднее значение точности
# за все итерации в эпохе.
# В качестве оптимизатора используется Adam.
# Защита от переобучения dropout + техника ранней остановки
def train(X_train, y_train, X_test, y_test) :
    tf.reset_default_graph()
    i = 1
    X = tf.placeholder(tf.float32, shape=(None, 88, 88), name="input")
    y = tf.placeholder(tf.int32, shape=(None, 8), name="labels")
    
    dropout = tf.placeholder(tf.float32, name="dropout")
    
    logits = cnn_model(X, dropout)
    
    with tf.name_scope("loss") :
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits))
        
    with tf.name_scope("train") :
        optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
        train_step = optimizer.minimize(loss)
        
    with tf.name_scope("accuracy") :
        predicted = tf.argmax(logits, 1, name="prediction")
        truth = tf.argmax(y, 1)
        classif = tf.nn.softmax(logits, name="classification")
        correct_prediction = tf.equal(predicted, truth)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
        
    with tf.Session() as sess :
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer())
        print("Поехали\n")
        best_acc = 0
        number = len(X_train)
        start_time = time.time()
        for i in range(epoch) :
            X_train, y_train = shuffle(X_train, y_train, random_state=42)
            for l in range(0, number, batch_size) :
                end = l + batch_size
                X_batch, y_batch = X_train[l:end], y_train[l:end]
                sess.run(train_step, feed_dict = {X: X_batch, y: y_batch, dropout: 0.5})
                
            num_examples = len(X_test)
            total_accuracy = 0
            for k in range(0, num_examples,batch_size) :
                X_batchT, y_batchT = X_test[k:k+batch_size], y_test[k:k+batch_size]
                res_accuracy = sess.run(accuracy, feed_dict={X: X_batchT, y: y_batchT, dropout:0})
                total_accuracy += (res_accuracy * len(X_batchT))
                
            #Техника ранней остановки - если нейронная сеть на проверочных данных не показывает
            #лучший результат в течении некоторого количества шагов(max_checks_without_progress)
            #то сохраняется модель с наилучшим результатом точности
            
            if total_accuracy / num_examples > best_acc :
                i+=1
                saver.save(sess, "./bestmodel")
                best_acc = total_accuracy / num_examples
                checks_without_saves = 0
            else:
                checks_without_saves+=1
                
            if checks_without_saves > max_checks_without_progress:
                print("Early stopping!")
                break
            print("Epoch:", i, " best accuracy:",
                  best_acc * 100, " current accuracy:", (total_accuracy / num_examples) * 100)
        print("\nTotal training time", time.time()-start_time)

In [32]:
# Оценка готовой модели, также сюда передается набор тестовых данных и разметки,
# передается путь к модели для проверки в этот метод,
# вид этого пути "..../bestmodel".
# Этот метод выводит среднюю ошибку по всему набору проверочных данных
def evaluate(X_test, y_test, path_to_model) :
    tf.reset_default_graph()
    saver = tf.train.import_meta_graph(path_to_model+".meta")
    with tf.Session() as sess :
        saver.restore(sess, tf.train.latest_checkpoint('./'))
        graph = tf.get_default_graph()
        num_examples = len(X_test)
        accuracy = graph.get_tensor_by_name("accuracy/accuracy:0")
        X = graph.get_tensor_by_name("input:0")
        y = graph.get_tensor_by_name("labels:0")
        dropout = graph.get_tensor_by_name('dropout:0')
        total_accuracy = 0
        for k in range(0, num_examples,batch_size) :
            X_batchT, y_batchT = X_test[k:k+batch_size], y_test[k:k+batch_size]
            res_accuracy = sess.run(accuracy, feed_dict={X: X_batchT, y: y_batchT, dropout:0})
            total_accuracy += (res_accuracy * len(X_batchT))
        print(total_accuracy / num_examples * 100)

In [8]:
# Метод классификации принимает 
# принимает на вход путь к изображени и путь к модели
# пример пути к изображени:"./picture.jpg"
# пример пути к модели "..../bestmodel".
# метод выводит класс, к которому принадлежит картинка и вероятности принадлежности
# ко всем классам.
def classification(path_to_img, path_to_model) :
    tf.reset_default_graph()
    test_img = converting_img(path_to_img)
    saver = tf.train.import_meta_graph(path_to_model+".meta")
    with tf.Session() as sess :
        saver.restore(sess, tf.train.latest_checkpoint('./'))
        graph = tf.get_default_graph()
        predicted = graph.get_tensor_by_name("accuracy/prediction:0")
        class_probalities = graph.get_tensor_by_name("accuracy/classification:0")
        X = graph.get_tensor_by_name("input:0")
        dropout = graph.get_tensor_by_name('dropout:0')
        class_prob, pred = sess.run([class_probalities, predicted], feed_dict={X: test_img, dropout:0})                 
        print("digit:", pred+1, "probabilities:", class_prob)

In [10]:
#Метод подготовки картинки для класификации
def converting_img(path) :
    with tf.Session() as sess:
        img = []
        convert = tf.image.rgb_to_grayscale(plt.imread(path))
        conv_img = sess.run(convert)
        resize = tf.image.resize_images(conv_img, [88, 88])
        pre_img = np.squeeze(sess.run(resize), 2)
        img.append(pre_img)
    return img

In [5]:
#Загрузка данных из уже готовых массивов numpy
#X_train, y_train, X_test, y_test = load_prepared_data(data_path)

In [34]:
#Обучение нейронной сети
#train(X_train, y_train, X_test, y_test)

<class 'tensorflow.python.framework.ops.Tensor'>
Поехали

Epoch: 1  best accuracy: 63.20653375363877  current accuracy: 63.20653375363877
Epoch: 2  best accuracy: 76.61723607641483  current accuracy: 76.61723607641483
Epoch: 3  best accuracy: 85.72963686520224  current accuracy: 85.72963686520224
Epoch: 3  best accuracy: 85.72963686520224  current accuracy: 84.05329874836191
Epoch: 5  best accuracy: 92.77885224315496  current accuracy: 92.77885224315496
Epoch: 6  best accuracy: 93.91790212110219  current accuracy: 93.91790212110219
Epoch: 7  best accuracy: 96.41091846891969  current accuracy: 96.41091846891969
Epoch: 8  best accuracy: 96.64732468222998  current accuracy: 96.64732468222998
Epoch: 8  best accuracy: 96.64732468222998  current accuracy: 95.89512187538418
Epoch: 10  best accuracy: 97.61444379789917  current accuracy: 97.61444379789917
Epoch: 11  best accuracy: 97.76488372184578  current accuracy: 97.76488372184578
Epoch: 11  best accuracy: 97.76488372184578  current accurac

KeyboardInterrupt: 

In [35]:
#Оценка точности модели на проверочном наборе
#evaluate(X_test, y_test, r'C:\Users\pyroman\task\bestmodel')

INFO:tensorflow:Restoring parameters from ./bestmodel
98.7320016807775


In [12]:
#Классификация одного рисунка
#classification("C:/Users/pyroman/1.jpg" ,"C:/Users/pyroman/task/bestmodel")

ValueError: Can't load save_path when it is None.

In [3]:
!jupyter nbconvert --to script task.ipynb

[NbConvertApp] Converting notebook task.ipynb to script
[NbConvertApp] Writing 15643 bytes to task.py


In [2]:
!python task.py


digit: [7] probabilities: [[6.8942668e-14 1.0150678e-13 1.4211731e-05 3.5834855e-03 7.3180503e-11
  9.1218457e-02 9.0517962e-01 4.1785834e-06]]


C:\Program Files11\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Traceback (most recent call last):
  File "task.py", line 467, in <module>
    get_ipython().system('jupyter nbconvert --to script task.ipynb')
NameError: name 'get_ipython' is not defined
2018-08-07 11:26:30.360320: I C:\users\nwani\_bazel_nwani\mmtm6wb6\execroot\org_tensorflow\tensorflow\core\platform\cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX
2018-08-07 11:26:30.711425: I C:\users\nwani\_bazel_nwani\mmtm6wb6\execroot\org_tensorflow\tensorflow\core\common_runtime\gpu\gpu_device.cc:1356] Found device 0 with properties: 
name: GeForce GTX 1050 major: 6 minor: 1 memoryClockRate(GHz): 1.455
pciBusID: 0000:01:00.0

In [71]:
type(str(1))

str

In [14]:
str([1,2,3])

'[1, 2, 3]'